A tener en cuenta:

TS: falta automatizar deteccion de cantidad de planillas en el archivo. falta documentar
dexa: Ok proceso de descarga. falta documentar
Selnet: EL proceso esta OK, tengo problemas para cambiarle el nombre al archivo (se rompe si lo hago)

In [18]:
import time
import requests
from bs4 import BeautifulSoup

In [4]:
#Extract TS, documentar!!! (planilla en Drive)
#automatizar deteccion de cantidad de planillas en el archivo

In [5]:
# Configurar las credenciales del servicio
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('listadeprecios-rocnet-2ebb9564fd3b.json', scope)

# Autenticación con Google
client = gspread.authorize(credentials)

spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1E9oFbOPCYIEqZO7rOOC9Qbk46HwgGO-V_qwLaww5USY/edit#gid=1458627182'
spreadsheet = client.open_by_url(spreadsheet_url)

# Crear un diccionario para almacenar los dataframes
dfs = {}

for worksheet_name in ['HIKVISION', 'ALARMA HIK', 'EZVIZ + IMOU', 'DAHUA', 'ACCESORIOS/VARIOS']:
    worksheet = spreadsheet.worksheet(worksheet_name)
    data = worksheet.get_all_values()
    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)
    df_name = worksheet_name.lower().replace(' ', '_') + '_base'  # Definir el nombre del dataframe
    dfs[df_name] = df

TS_hikvision_df = dfs['hikvision_base']
TS_alarmahik_df = dfs['alarma_hik_base']
TS_ezviz_imou_df = dfs['ezviz_+_imou_base']
TS_dahua_df = dfs['dahua_base']
TS_accesorios_varios_df = dfs['accesorios/varios_base']

In [6]:
TS_hikvision_df

,10%,,,,,
0,,,,,,
1,,,,,,
2,,B110P,Camara Infrarroja HILOOK | Exterior |Plastica ...,$5.600,,
3,,B110M,Camara Infrarroja HILOOK | Exterior |Metal | 4...,$5.800,,
4,,T110M,Camara Infrarroja HILOOK | IExterior | Metal |...,$5.800,,
...,...,...,...,...,...,...
142,,DS-KD-KP,Estación de puerta modular HIKVISION | Exterio...,$50.000,,ULTIMAS UNIDADES
143,,CENTRAL P VSS BASE HW/64CH,Servidor HIKVISION | Vista en vivo y reproducc...,$677.000,,ULTIMAS UNIDADES
144,,MERCHADISING,,,,
145,,GORRAH,GORRA HIKVISION | AJUSTABLES | ESTAMPADO EN VI...,$1.300,,


In [5]:
import json

In [6]:
with open("credenciales.json") as archivo:
    credenciales = json.load(archivo)

In [16]:
import selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import os
from datetime import datetime

In [17]:
user = credenciales["user"]
password = credenciales["password"]

# Obtén el directorio de trabajo actual
project_folder = os.getcwd()
download_folder = os.path.join(project_folder, "C:\\Users\\Juan M Cabral\\Desktop\\Pycharm\\lista_de_precios_axon\\rawdata")

# Verifica si la carpeta de descargas existe, si no, créala
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Configurar el Webdriver
options = Options()
options.add_experimental_option("detach", True)
prefs = {
    "download.default_directory": download_folder
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

driver.get("https://www.dexa.net.ar/productos/catalogo")
driver.implicitly_wait(3)

user_input = driver.find_element(by=By.NAME, value="data[Usuario][login]")
user_input.send_keys(user)

user_input = driver.find_element(by=By.NAME, value="data[Usuario][password]")
user_input.send_keys(password)

driver.find_element(by=By.XPATH, value='//button[@class="btn btn-primary" and text()="Ingresar"]').click()
time.sleep(4)

dropdown = driver.find_element(by=By.LINK_TEXT, value='Productos')

# Hacer clic en el elemento de lista desplegable
dropdown.click()

# Esperar a que aparezcan las opciones del menú desplegable
time.sleep(3)

driver.find_element(by=By.XPATH, value='//a[contains(text(), "Exportar lista de precios")]').click()
time.sleep(3)
driver.find_element(By.XPATH, '//button[contains(.,"Descargar lista de precios")]').click()
time.sleep(15)

fecha_actual = datetime.now().strftime("%d_%m_%Y")

# Especificar el nombre del archivo descargado
nombre_archivo_descargado = "Lista Precios DEXA Seguridad Electrónica S.R.L. {}.xlsx".format(fecha_actual)

# Esperar a que el archivo se descargue completamente
while not os.path.exists(os.path.join(download_folder, nombre_archivo_descargado)):
    time.sleep(1)

# Obtener la ruta completa del archivo descargado
downloaded_file_path = os.path.join(download_folder, nombre_archivo_descargado)

# Renombrar el archivo descargado a "dexa.xlsx"
nuevo_nombre_archivo = "dexa.xlsx"
nueva_ruta_archivo = os.path.join(download_folder, nuevo_nombre_archivo)
os.rename(downloaded_file_path, nueva_ruta_archivo)



In [ ]:
#CONFIGURAR EL PATH Y NOMBRE PARA LA DESCARGA DEL ARCHIVO!

In [29]:

import zipfile

# Ruta de la carpeta de destino
download_folder = "C:\\Users\\Juan M Cabral\\Desktop\\Pycharm\\lista_de_precios_axon\\rawdata"

# URL del archivo zip en Dropbox
url = "https://www.dropbox.com/sh/c1y13dqqnqwgpfk/AADmt_8B5LUPatOkehCuIfrTa?dl=1"

# Realizar la solicitud GET para obtener el contenido del archivo zip
response = requests.get(url)

# Guardar el contenido del archivo zip en disco
file_path = os.path.join(download_folder, "archivo.zip")
with open(file_path, "wb") as file:
    file.write(response.content)

# Descomprimir el archivo zip en la carpeta de destino
with zipfile.ZipFile(file_path, "r") as zip_ref:
    zip_ref.extractall(download_folder)



'C:\\Users\\Juan M Cabral\\Desktop\\Pycharm\\lista_de_precios_axon\\rawdata\\selnet.xlsx'

In [ ]:
#esto ya esta qe no sirve mas, arreglar!

In [21]:
def cargar_datos(ruta_del_archivo, prefijo):
    archivo_excel = pd.ExcelFile(ruta_del_archivo)
    nombres_hojas = archivo_excel.sheet_names

    for hoja in nombres_hojas:
        # Lee el DataFrame correspondiente a cada hoja
        dataframe = archivo_excel.parse(hoja)
        # Limpia el nombre de la hoja eliminando caracteres problemáticos y espacios
        nombre_hoja_limpio = re.sub(r'[^a-zA-Z0-9_]', '', hoja)
        # Asigna un nombre específico al DataFrame utilizando el prefijo y el nombre limpio de la hoja
        nombre_df = f"{prefijo}_{nombre_hoja_limpio.lower()}_df"
        # Crea una variable con el nombre específico y asigna el DataFrame correspondiente
        globals()[nombre_df] = dataframe
        print(nombre_df)

# Rutas de los archivos
rutas_archivos = [
    '/content/drive/MyDrive/Colab Notebooks/Axon lista de precios/Lista Precios DEXA Seguridad Electrónica S.R.L. 06_06_2023.xlsx',
    '/content/drive/MyDrive/Colab Notebooks/Axon lista de precios/Lista OCRL GREMIO.xlsx',
    '/content/drive/MyDrive/Colab Notebooks/Axon lista de precios/Lista de Precios Tayler `22.xlsx',
    '/content/drive/MyDrive/Colab Notebooks/Axon lista de precios/Lista de precios - GREMIO (1).xlsx'
]

# Prefijos para los DataFrames
prefijos = ['dexa', 'ocrl', 'tayler', 'selnet']

# Cargar los datos de cada archivo utilizando los prefijos correspondientes
for ruta, prefijo in zip(rutas_archivos, prefijos):
    cargar_datos(ruta, prefijo)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Axon lista de precios/Lista Precios DEXA Seguridad Electrónica S.R.L. 06_06_2023.xlsx'